# SAP-1 Loader using firmata
The code is read from a .txt file in the same folder and it must not have empty lines

In [66]:
program_file = 'triangular.txt'
program = {'CODE' : [], 'VALUE' : []}

assembly = {'NOP': 0x0, 'LDA': 0x1, 'ADD': 0x2,
             'SUB': 0x3, 'STA': 0x4, 'LDI': 0x5,
             'JMP': 0x6, 'JC':0x7, 'JZ':0x8,
             'OUT': 0xE, 'HLT':0xF}

PROGRAM_LEN = 16


### Reading and interpreting the code

In [67]:
def printProgram():
  for i in range(PROGRAM_LEN):
    print(f'0x{i:0X} : {program["CODE"][i]} \t 0x{program["VALUE"][i]:02X} \t {program["VALUE"][i]:08b} ')



with open(program_file) as f:
  lines = f.readlines()
  while len(lines) < PROGRAM_LEN:
    lines.append('0 \n') #Append zeros if there are missing lines

  for l in lines:
    l = l.strip() # Removing any white space and newline
    data = l.split()  #split command and values

    if not assembly.get(data[0]) == None: #If it is in the dictionary it is a command

      if data[0]=='NOP' or data[0]=='HLT' or data[0]=='OUT': #Commands without values (0 by default)
        program['CODE'].append(f'{data[0]} 0x{0:0X}')
        program['VALUE'].append(assembly[data[0]]*0x10 + 0x0)
      else:                                                   #Commands with a value
        value = int(data[1], PROGRAM_LEN)  
        program['CODE'].append(f'{data[0]} 0x{value:0X}')
        program['VALUE'].append(assembly[data[0]]*0x10 + value)
    
    else:                                                    #Memory data, without a command
      value = int(data[0], PROGRAM_LEN)
      program['CODE'].append(f'0x{value:02X}')
      program['VALUE'].append(value)

printProgram()


0x0 : LDI 0x1 	 0x51 	 01010001 
0x1 : STA 0xE 	 0x4E 	 01001110 
0x2 : LDI 0x0 	 0x50 	 01010000 
0x3 : OUT 0x0 	 0xE0 	 11100000 
0x4 : ADD 0xE 	 0x2E 	 00101110 
0x5 : JC 0x0 	 0x70 	 01110000 
0x6 : STA 0xF 	 0x4F 	 01001111 
0x7 : LDI 0x1 	 0x51 	 01010001 
0x8 : ADD 0xE 	 0x2E 	 00101110 
0x9 : STA 0xE 	 0x4E 	 01001110 
0xA : LDA 0xF 	 0x1F 	 00011111 
0xB : JMP 0x3 	 0x63 	 01100011 
0xC : 0x00 	 0x00 	 00000000 
0xD : 0x00 	 0x00 	 00000000 
0xE : 0x00 	 0x00 	 00000000 
0xF : 0x00 	 0x00 	 00000000 


### Arduino part

In [68]:
import pyfirmata
import time

board = pyfirmata.Arduino('/dev/ttyUSB0') #Change the port if the Arduino doesn't show up
print('Communication started')

address_pin = [board.digital[i] for i in [8,7,6,5]] #Pins for the memory address
for pin in address_pin: pin.write(0)

data_pin = board.digital[2]      #Shift register pins
clock_pin = board.digital[3]
latch_pin = board.digital[4]

button_pin = board.digital[13]  #Pin that copies the temporary data inside the register, active LOW
button_pin.write(1)


Communication started


### Arduino functions

In [69]:
def set_address(address):
  '''
    Function to set the address
  '''
  for i in range(4):
    bit = (address >> i) & 1
    address_pin[i].write(bit)

def send_byte(byte, bitOrder = 'MSBFIRST'): 
  '''
    Function to send a byte 
  '''
  latch_pin.write(0)  #Latch low so the writing doesn't affect the output
  for i in range(8):
    if (bitOrder == 'LSBFIRST'): #LSBFIRST
      data_pin.write(byte >> i & 1) #Write the bit to the serial data line
    else:                        #MSBFIRST
      data_pin.write(byte >> 7-i & 1) 

    clock_pin.write(1)  #Clock signal to shift the data
    clock_pin.write(0) 

  latch_pin.write(1)  #Latch high to change the output
  time.sleep(0.01)  #Wait 10 ms
  latch_pin.write(0)  #Latch low


## Write the program by writing all the lines

In [70]:
for i in range(PROGRAM_LEN):

  set_address(i)
  print(f'Accessing memory address {i}: {i:04b}')
  send_byte(program['VALUE'][i], 'LSBFIRST')
  print(f'\tWriting the instruction {program["CODE"][i]} : {program["VALUE"][i]:08b} ')

  button_pin.write(0) #copy the value in the memory
  time.sleep(0.20)
  button_pin.write(1)

  time.sleep(0.1) #Wait before sending another line


Accessing memory address 0: 0000
	Writing the instruction LDI 0x1 : 01010001 
Accessing memory address 1: 0001
	Writing the instruction STA 0xE : 01001110 
Accessing memory address 2: 0010
	Writing the instruction LDI 0x0 : 01010000 
Accessing memory address 3: 0011
	Writing the instruction OUT 0x0 : 11100000 
Accessing memory address 4: 0100
	Writing the instruction ADD 0xE : 00101110 
Accessing memory address 5: 0101
	Writing the instruction JC 0x0 : 01110000 
Accessing memory address 6: 0110
	Writing the instruction STA 0xF : 01001111 
Accessing memory address 7: 0111
	Writing the instruction LDI 0x1 : 01010001 
Accessing memory address 8: 1000
	Writing the instruction ADD 0xE : 00101110 
Accessing memory address 9: 1001
	Writing the instruction STA 0xE : 01001110 
Accessing memory address 10: 1010
	Writing the instruction LDA 0xF : 00011111 
Accessing memory address 11: 1011
	Writing the instruction JMP 0x3 : 01100011 
Accessing memory address 12: 1100
	Writing the instruction 0x00